In [1]:
cd ..

/Users/kimyoungh/GoogleDrive/rich/main/rich


In [2]:
import datetime
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from data_processing.trading_gpt_data_generator import TradingGPTDataGenerator
from trading.kis_data_loader import KISDataLoader

%load_ext autoreload
%autoreload 2

In [3]:
loader = KISDataLoader()

In [4]:
today = datetime.datetime.today().strftime('%Y%m%d')

In [5]:
k200 = loader.get_timeseries(
    '069500', start_date='20041230',
    end_date='20230426', period='D')

In [6]:
k200

,open,high,low,close,volume,turnover
stck_bsop_date,,,,,,
20041230,8260.0,8440.0,8260.0,8440.0,2267109.0,1.896828e+10
20050103,8440.0,8468.0,8361.0,8418.0,1309079.0,1.100016e+10
20050104,8361.0,8440.0,8361.0,8375.0,683869.0,5.744800e+09
20050105,8332.0,8375.0,8260.0,8375.0,1248785.0,1.039094e+10
20050106,8353.0,8382.0,8282.0,8296.0,914020.0,7.610162e+09
...,...,...,...,...,...,...
20230420,33278.0,33387.0,33170.0,33293.0,6320778.0,2.105694e+11
20230421,33219.0,33283.0,33007.0,33125.0,4982093.0,1.651558e+11
20230424,33017.0,33066.0,32874.0,32898.0,4989862.0,1.645324e+11


In [7]:
k200 = k200.rename(columns={'turnover': 'value'})

In [10]:
pos = np.argwhere(k200.index == '20181228').item()

In [11]:
pos

3463

In [12]:
window = 250
gen = TradingGPTDataGenerator(k200.iloc[:pos+1], window=window)
gen_test = TradingGPTDataGenerator(k200.iloc[-(pos-window):], window=window)
#gen_test = TradingGPTDataGenerator(k200.iloc[3709-250:], window=250)
gen_overall = TradingGPTDataGenerator(k200, window=window)

In [13]:
datasets = gen.generate_dataset()
datasets_test = gen_test.generate_dataset()
datasets_overall = gen_overall.generate_dataset()

In [14]:
print(datasets['observations'].shape)
print(datasets_test['observations'].shape)
print(datasets_overall['observations'].shape)

(3214, 8)
(2963, 8)
(4282, 8)


In [15]:
print(datasets['returns'].shape)
print(datasets_test['returns'].shape)
print(datasets_overall['returns'].shape)

(3214,)
(2963,)
(4282,)


In [16]:
datasets_overall.keys()

dict_keys(['observations', 'returns', 'timestamps'])

In [17]:
datasets_overall['timestamps'].shape

(4282,)

In [18]:
datasets['observations']

array([[0.08148148, 0.62222222, 1.        , ..., 1.        , 0.29559094,
        0.        ],
       [0.99999999, 0.57894737, 1.        , ..., 1.        , 0.37265342,
        1.        ],
       [0.72602739, 0.8219178 , 0.98303432, ..., 1.        , 0.59714018,
        1.        ],
       ...,
       [0.60126582, 0.9113924 , 0.05108173, ..., 0.05428571, 0.55242746,
        1.        ],
       [0.60507246, 0.50724637, 0.015625  , ..., 0.00736842, 0.21912615,
        0.        ],
       [0.51190476, 0.35119047, 0.05979567, ..., 0.05157895, 0.30523431,
        1.        ]])

In [19]:
datasets['returns']

array([ 0.00498942,  0.00052655,  0.00631531, ..., -0.01302333,
        0.01243392,  0.00417728])

In [20]:
k200.to_parquet('train_data/trading_gpt/price_train_k200.pq')

In [21]:
with open('train_data/trading_gpt/train_dataset_k200.pkl', 'wb') as f:
    pickle.dump(datasets, f)

with open('train_data/trading_gpt/test_dataset_k200.pkl', 'wb') as f:
    pickle.dump(datasets_test, f)

with open('train_data/trading_gpt/overall_dataset_k200.pkl', 'wb') as f:
    pickle.dump(datasets_overall, f)